In [1]:
from transformers import ElectraModel, ElectraTokenizer

In [22]:
tokenizer = ElectraTokenizer.from_pretrained('monologg/koelectra-small-v3-discriminator')

Downloading:   0%|          | 0.00/257k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/61.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/458 [00:00<?, ?B/s]

In [3]:
from haystack.utils import clean_wiki_text, convert_files_to_docs, fetch_archive_from_http, print_answers
from haystack.nodes import FARMReader, TransformersReader

INFO - haystack.document_stores.base -  Numba not found, replacing njit() with no-op implementation. Enable it with 'pip install numba'.
INFO - haystack.modeling.model.optimization -  apex not found, won't use it. See https://nvidia.github.io/apex/
ERROR - root -  Failed to import 'magic' (from 'python-magic' and 'python-magic-bin' on Windows). FileTypeClassifier will not perform mimetype detection on extensionless files. Please make sure the necessary OS libraries are installed if you need this functionality.


In [4]:
from haystack.document_stores import InMemoryDocumentStore

In [5]:
document_store = InMemoryDocumentStore()

INFO - haystack.modeling.utils -  Using devices: CPU
INFO - haystack.modeling.utils -  Number of GPUs: 0


In [6]:
import os
from subprocess import Popen, PIPE, STDOUT

In [70]:
document_store.delete_index

<bound method ElasticsearchDocumentStore.delete_index of <haystack.document_stores.elasticsearch.ElasticsearchDocumentStore object at 0x0000026529812AC0>>

In [71]:
from haystack.document_stores import ElasticsearchDocumentStore
document_store = ElasticsearchDocumentStore(host='localhost', username='root', password='1111', index='document')

C:\Users\Nayoun Kim\AppData\Roaming\Python\Python39\site-packages\elasticsearch\connection\base.py:190: ElasticsearchDeprecationWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.15/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchDeprecationWarning)


In [8]:
import pandas as pd
newsdf = pd.read_csv('../data/news_double.csv')

In [9]:
newsdf.head()

,title,article,link,sub
0,"꼬박 국민연금 부어도 57만원, 기초연금 부부 64만원…""속터져""",“국민연금은 용돈 수준인데 기초연금 월 40만원 시대가 열린다니 여태껏 착하게 연금을 부어 온 이들이 화날 것 같다.” 최근 인...,https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=102&oid=025&aid...,개인연금
1,"꼬박 국민연금 부어도 57만원, 기초연금 부부 64만원…""속터져""",서울 국민연금공단 송파지사 상담 창구의 모습. 연합뉴스 그러나 기초연금을 인상하면 연금 가입 유인이 떨어지고 가입자가 ...,https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=102&oid=025&aid...,개인연금
2,"꼬박 국민연금 부어도 57만원, 기초연금 부부 64만원…""속터져""",기초연금 관련 이미지. 연합뉴스 연금공단 관계자는 “상담 과정에서 거의 100이면 100 기초연금이 안 깎이려면 어떻게 해야 ...,https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=102&oid=025&aid...,개인연금
3,5년만에 0명→2472명…국민연금 월 200만원 '황금 황혼' 누구? [뉴스원샷],국민연금을 월 200만원 이상 수령하는 사람이 2500명에 육박하는 것으로 나타났다. 5년 전엔 단 한명도 없었는데 최근 급증했다. ...,https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=102&oid=025&aid...,개인연금
4,"기초연금 40만원 시대, 주부가 국민연금 가입 전에 따져볼 3가지","“내 노후는 내가 알아서 준비하겠다, 지금까지 낸 돈 이자는 안 받을 테니 원금만이라도 돌려 달라.”강제로 가입하는 국민연금에 대한 뉴스...",https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=101&oid=023&aid...,개인연금


In [110]:
news_dicts = []
for i in range(len(newsdf['title'])):
      data = newsdf.iloc[i]
      temp = {}
      short_article = data['article'].strip()
      temp['content'] = short_article
      temp['meta'] = {'title': data['title'], 'subject': data['sub']}
      news_dicts.append(temp)

In [111]:
news_dicts[:3]

[{'content': '“국민연금은 용돈 수준인데 기초연금 월 40만원 시대가 열린다니 여태껏 착하게 연금을 부어 온 이들이 화날 것 같다.”      최근 인터넷 블로그에 이런 글이 올라왔다. 새 정부가 앞으로 65세 이상 소득 낮은 70% 노인에게 주는 기초연금을 10만원 많은 40만원으로 인상하겠다고 발표한 걸 두고서다.      기초연금 40만원 시대가 현실화할 경우 대상이 되는 부부는 감액(20%)을 적용받아 64만원을 타게 된다. 한 가정이 꼬박 보험료를 내서 손에 쥐는 국민연금 평균액(57만원)보다 많게 되니 상대적 박탈감을 느낄 수 있다. 굳이 국민연금을 타고자 의무가입 기간을 채울 필요가 있느냐는 의문도 나온다.',
  'meta': {'title': '꼬박 국민연금 부어도 57만원, 기초연금 부부 64만원…"속터져"',
   'subject': '개인연금'}},
 {'content': '서울 국민연금공단 송파지사 상담 창구의 모습. 연합뉴스           그러나 기초연금을 인상하면 연금 가입 유인이 떨어지고 가입자가 이탈할 것이란 지적은 과하다고 전문가들은 말한다.       오건호 내가만드는복지국가 정책위원장은 “기초연금과 국민연금은 양자택일이 아니기 때문에 기초연금이 오르면 국민연금을 회피할 것이란 주장은 근거가 약하다”라고 했다. 만 18세 이상에서 만 60세 미만 국민 중 소득이 있으면 공무원연금 등 특별한 경우를 제외하고는 국민연금에 의무로 가입해야 한다. 본인 의사에 따라 가입할 수 있는 게 아니란 얘기다.       기초연금은 자격 요건이 되면 받는 것이다. 오 위원장은 “사적 개인연금처럼 가입자가 선택할 것처럼 가정하고 하는 얘기인데 오해와 불신에서 비롯된 것”이라며 “두 개 다 받을 수 있으면 그렇게 하는 게 이득”이라고 말했다.       다만 국민연금 사각지대가 존재한다는 점을 고려하면 전체 가입자는 아니더라도 당장 연금 보험료를 내기 부담스러울 일부 저소득 영세 자영업자의 장기 가입 유인이 떨어질 수는 있다고 전문가

In [109]:
# document_store.delete_documents()
# document_store.describe_documents()

C:\Users\Nayoun Kim\AppData\Roaming\Python\Python39\site-packages\elasticsearch\connection\base.py:190: ElasticsearchDeprecationWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.15/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchDeprecationWarning)
C:\Users\Nayoun Kim\anaconda3\envs\silverfund\lib\site-packages\numpy\core\fromnumeric.py:3474: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\Nayoun Kim\anaconda3\envs\silverfund\lib\site-packages\numpy\core\_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


ValueError: max() arg is an empty sequence

In [112]:
document_store.write_documents(news_dicts)

In [113]:
from haystack.nodes import BM25Retriever
retriever = BM25Retriever(document_store=document_store)

In [141]:
reader = TransformersReader(model_name_or_path='monologg/koelectra-small-v2-distilled-korquad-384', tokenizer='monologg/koelectra-small-v2-discriminator', context_window_size=800)

INFO - haystack.modeling.utils -  Using devices: CPU
INFO - haystack.modeling.utils -  Number of GPUs: 0


In [177]:
from haystack.pipelines import ExtractiveQAPipeline

pipe = ExtractiveQAPipeline(reader, retriever)

In [143]:
prediction = pipe.run(
    query = '국민연금 적립금은 언제 소진되나요?',
    params={"Retriever": {"top_k": 10}, "Reader": {"top_k": 1}}
)

C:\Users\Nayoun Kim\AppData\Roaming\Python\Python39\site-packages\elasticsearch\connection\base.py:190: ElasticsearchDeprecationWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.15/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchDeprecationWarning)
C:\Users\Nayoun Kim\anaconda3\envs\silverfund\lib\site-packages\transformers\tokenization_utils_base.py:707: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  tensor = as_tensor(value)
C:\Users\Nayoun Kim\anaconda3\envs\silverfund\lib\site-packages\transformers\pipelines\question_answering.py:297: VisibleDeprecationWarning: Creating an n

In [181]:
from haystack.utils import print_answers

prediction = pipe.run(
     query = '국민연금 적립금은 언제 소진되나요?',
    params={"Retriever": {"top_k": 10}, "Reader": {"top_k": 1}}
)
print_answers(prediction, details='medium')


Query: 국민연금 적립금은 언제 소진되나요?
Answers:
[   {   'answer': '2055년',
        'context': '자면 국민연금은 노후소득 보장의 핵심 기둥이고, 반드시 지급되어야 한다. 우리가 논의해야 할 것은 그 '
                   '책임을 누가 언제 어떻게 나눠질 것인가의 문제다. 재정의 지속가능성, 높은 노인빈곤율에 대응하는 노후소득 '
                   '보장 강화는 선후의 문제가 아니라 함께 해결해야 할 과제다.연금이 고차방정식인 이유는 현재와 미래의 '
                   "문제를 함께 다뤄야 하기 때문이다. 흔히들 세대 문제라고 하는데 이때 세대는 '노인과 청년' 둘이 "
                   '아니다. 현재 노인인 세대, 미래에 노인이 될 현재의 청장년 세대, 현재의 아동과 아직 태어나지 않은 '
                   '아이들로 구성될 미래의 청장년 세대 모두 관련되어 있다. 따라서 국민연금은 우리세대와 미래세대의 공존을 '
                   '상징하는 제도이기도 하다.재정 건전성 유지의 필요성현행 국민연금 제도는 받는 급여에 비해 내는 기여가 '
                   '낮아 수지불균형이 무척 크다. 게다가 인구는 빠르게 줄어들고 고령화는 급속도로 진행되고 있다. 지금 '
                   '상태를 그대로 두면 미래세대는 수지불균형 문제와 초고령화 부담을 함께 지게 된다. 국민연금법은 이런 '
                   '문제를 해결하기 위해 장기적으로 재정 균형을 유지할 수 있도록 급여 수준과 연금보험료를 조정하라고 법에 '
                   '명시하고 있다.재정 계산은 5년에 한 번 이뤄지고 현재의 법·제도가 유지된다는 가정하에 70년 전망치를 '
                   '제시한다. 2003년 이후 4차례 실시됐고, 올해 5차 재정 추계가 시작될 예정이다.